In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec17.txt'

In [2]:
WALL = ord('#')
EMPTY = ord('.')
INTERSECTION = ord('O')

class InputWrapper:
    def __init__(self, robot):
        self.robot = robot
    
    def get(self):
        return None

class GetBoardOutputWrapper:
    def __init__(self, robot, input_wrapper):
        self.robot = robot
        self.input_wrapper = input_wrapper
        self.pos_x = 0
        self.pos_y = 0
    
    def __call__(self, val):
        if val == ord('\n'):
            self.pos_y += 1
            self.pos_x = 0
        else:
            self.robot.painted[(self.pos_x, self.pos_y)] = val
            self.pos_x += 1
        print(chr(val), end='')
            
class Robot:
    def __init__(self, ops):
        self.ops = ops
        self.painted = defaultdict(lambda: EMPTY)

    def simulate(self):
        input_wrapper = InputWrapper(self)
        output_wrapper = GetBoardOutputWrapper(self, input_wrapper)
        sim.simulate(self.ops, inputs=input_wrapper, output_func=output_wrapper)
        return 
    
    def printme(self):
        min_x = min(key[0] for key in self.painted.keys())
        max_x = max(key[0] for key in self.painted.keys())
        min_y = min(key[1] for key in self.painted.keys())
        max_y = max(key[1] for key in self.painted.keys())
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                if self.painted[(x, y)] == EMPTY:
                    print('.', end='')
                elif self.painted[(x, y)] == WALL:
                    print('#', end='')
                else:
                    print(chr(self.painted[(x, y)]), end='')
            print() 
        

In [3]:
# Explore the space with DFS
robot = Robot(sim.read_ops(INPUT_PATH.read_text().strip()))
robot.simulate()

..........#########....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
......#####.......#....................................
......#...........#....................................
......#...........#....................................
......#...........#....................................
....#.#.....#######....................................
....#.#.....#..........................................
....#.#.....#..........................................
....#.#.....#..........................................
....#.#.....#..........................................
....#.#.....#...................................

In [4]:
def get_intersections(painted, inplace: bool = False):
    min_x = min(key[0] for key in painted.keys())
    max_x = max(key[0] for key in painted.keys())
    min_y = min(key[1] for key in painted.keys())
    max_y = max(key[1] for key in painted.keys())
    ints = set()
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            if painted[(x, y)] == WALL:
                if all(painted[foo] == WALL for foo in util.four_ways(x, y)):
                    if inplace:
                        painted[(x, y)] = INTERSECTION
                    ints.add((x, y))

    return ints

In [5]:
intersections = get_intersections(robot.painted, inplace=True)

In [6]:
robot.printme()

.........................................................
...........#########.....................................
...........#.......#.....................................
...........#.......#.....................................
...........#.......#.....................................
...........#.......#.....................................
...........#.......#.....................................
...........#.......#.....................................
...........#.......#.....................................
.......#####.......#.....................................
.......#...........#.....................................
.......#...........#.....................................
.......#...........#.....................................
.....#.#.....#######.....................................
.....#.#.....#...........................................
.....#.#.....#...........................................
.....#.#.....#...........................................
.....#.#.....#

In [7]:
print(f'The answer to part 1 is {sum(x * y for x, y in intersections)}')

The answer to part 1 is 6244


In [8]:
GO_STRAIGHT = 0
TURN_LEFT = 1
TURN_RIGHT = 2

UP = 10
DOWN = 20
LEFT = 30
RIGHT = 40

ch_to_dir = {'^': UP, '<': LEFT, '>': RIGHT, 'v': DOWN}

painted = {key: val for key, val in robot.painted.items() if val != EMPTY}

def _degree(painted, pos):
    if pos not in painted or painted[pos] == EMPTY:
        return -1
    return sum(v in painted and painted[v] != EMPTY for v in util.four_ways(*pos))

degree_one_spots = [
    key for key in painted if _degree(painted, key) == 1
]

start_pos = [pos for pos in degree_one_spots if chr(painted[pos]) in ch_to_dir][0]
start_direction = ch_to_dir[chr(painted[start_pos])]

end_pos = [pos for pos in degree_one_spots if pos != start_pos][0]

In [15]:
def turn_left(direction):
    if direction == UP:
        return LEFT
    if direction == LEFT:
        return DOWN
    if direction == DOWN:
        return RIGHT
    if direction == RIGHT:
        return UP
    raise ValueError('Bad direction')

def turn_right(direction):
    if direction == UP:
        return RIGHT
    if direction == LEFT:
        return UP
    if direction == DOWN:
        return LEFT
    if direction == RIGHT:
        return DOWN
    raise ValueError('Bad direction')

def step(pos, direction):
    if direction == UP:
        return (pos[0], pos[1] - 1)
    if direction == DOWN:
        return (pos[0], pos[1] + 1)
    if direction == LEFT:
        return (pos[0] - 1, pos[1])
    if direction == RIGHT:
        return (pos[0] + 1, pos[1])
    raise ValueError('Bad direction')

# Try always going straight through the intersections at first
def find_path(painted, start_pos, start_direction, end_pos):
    pos = start_pos
    direction = start_direction
    code = []
    while True:
        if pos == end_pos:
            return code
        
        next_pos = step(pos, direction)
        if next_pos in painted and painted[next_pos] != EMPTY:
            code.append(GO_STRAIGHT)
            pos = next_pos
            continue
        
        # Try to turn left
        left = turn_left(direction)
        next_pos = step(pos, left)
        if next_pos in painted and painted[next_pos] != EMPTY:
            code.append(TURN_LEFT)
            code.append(GO_STRAIGHT)
            pos = next_pos
            direction = left
            continue
        
        # Try to turn right
        right = turn_right(direction)
        next_pos = step(pos, right)
        if next_pos in painted and painted[next_pos] != EMPTY:
            code.append(TURN_RIGHT)
            code.append(GO_STRAIGHT)
            pos = next_pos
            direction = right
            continue
        
        raise ValueError()

In [17]:
code = find_path(painted, start_pos, start_direction, end_pos)

In [20]:
# Now try to create A, B, and C
MAX_WIDTH = 20

def compress(code):
    output = []
    while code:
        if code[0] == TURN_LEFT:
            output.append('L')
            code = code[1:]
        elif code[0] == TURN_RIGHT:
            output.append('R')
            code = code[1:]
        elif code[0] == GO_STRAIGHT:
            for i in range(1, len(code)):
                if code[i] != GO_STRAIGHT:
                    output.append(str(i))
                    code = code[i:]
                    break
            else:
                output.append(str(len(code)))
                code = []
        else:
            raise ValueError('boo')
    return ','.join(output)

def can_fit(path, a, b, c, output=None):
    output = output or []
    if not path:
        return output

    if len(output) + 2 > MAX_WIDTH:
        return False
    
    for inst, val in zip((a, b, c), range(3)):
        if all(x == y for x, y in zip(path, inst)):
            output.append(val)
            if can_fit(path[:len(inst)], a, b, c, output=output):
                return output
            output.pop()

In [28]:
MAX_CODES = 3

def find_instructions(path, codes=None, ccodes=None, output=None):
    #if ccodes and ccodes[-1] == 'R':
    #    return ccodes, output
    codes = codes or []
    ccodes = ccodes or []
    output = output or []

    if not path:
        return ccodes, output
    
    if 2 * len(output) + 1 > MAX_WIDTH:
        return False
    
    for code, val in zip(codes, range(len(codes))):
        if all(x == y for x, y in zip(path, code)):
            output.append(val)
            asdf = find_instructions(path[len(code):], codes=codes, ccodes=ccodes, output=output)
            if asdf:
                return asdf
            output.pop()
    
    if len(codes) < MAX_CODES:
        for end in range(1, len(path)):
            code = path[:end]
            ccode = compress(code)
            if len(ccode) > MAX_WIDTH:
                return False
            output.append(len(codes))
            codes.append(code)
            ccodes.append(ccode)
            asdf = find_instructions(path[len(code):], codes=codes, ccodes=ccodes, output=output)
            if asdf:
                return asdf
            output.pop()
            codes.pop()
            ccodes.pop()

routines, main = find_instructions(code)

In [39]:
# Explore the space with DFS
class AsciiOutput:
    def __call__(self, val):
        try:
            print(chr(val), end='')
        except:
            print(f'The answer to part 2 is {val}')

ops = sim.read_ops(INPUT_PATH.read_text().strip())
ops[0] = 2
inputs = '\n'.join([','.join('ABC'[i] for i in main)] + routines) + '\nn\n'
robot = sim.Computer(ops, inputs=[ord(x) for x in inputs], output_func=AsciiOutput())
robot.simulate()

..........#########....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
..........#.......#....................................
......#####.......#....................................
......#...........#....................................
......#...........#....................................
......#...........#....................................
....#.#.....#######....................................
....#.#.....#..........................................
....#.#.....#..........................................
....#.#.....#..........................................
....#.#.....#..........................................
....#.#.....#...................................

-1